Essential Packages

In [ ]:
%pip install  groq


In [ ]:
# pip install groq pdfplumber pytesseract watchdog python-dotenv pdf2image python-docx docx2txt 


In [ ]:
# %pip install "pip<24.1"
# %pip install textract


In [ ]:
%pip install --upgrade pymupdf

API connections

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")

In [ ]:
import os 
import groq

groq.api_key = GROQ_API_KEY

Extracting files

In [ ]:
import pdfplumber

In [ ]:
import groq

In [ ]:
from groq import Groq
client = Groq(api_key=groq.api_key)


summary_dict = {}

folder_queue = "resumes/queue"
for filename in os.listdir(folders):
    filepath = os.path.join(folders,filename)
    if os.path.isfile(filepath) and filename.lower().endswith((".pdf")):
        print("Processing file:", filename)
        doc = pdfplumber.open(f"/home/sneeraj/summariser/resumes/queue/{filename}")
        for page in doc.pages:
            text = page.extract_text() + "\n"
            # print(text)
            prompt = text + "\n tl;dr:"
            # print(len(text), text[:500])
            response = client.chat.completions.create(
                model="llama3-70b-8192",
                messages = [
                    {"role": "system",
                    "content" : (
                        "You are a AI assistant that extracts resume information from a resume  "
                        "always return json dont inclue any other text"
                        
                    )},
                    {"role": "user",
                    "content" : (
                        "first check if the pdf is valid or not and mark the status as valid or invalid if invalid u dont have to extract details just return the status in that json file"
                        f"Here is the resume \n\n {text}\n\n return this in json formate : \n and summarise at the end "
                            "{\n"
                            '  "status": "valid" or "invalid",\n'
                            '  "name": "",\n'
                            '  "email": "",\n'
                            '  "phone": "",\n'
                            '  "location": "",\n'
                            '  "education": [],\n'
                            '  "experience": [],\n'
                            '  "skills": []\n'
                            '  "summary": "",'
                            "}"

                    )
                    }
                ],
                temperature=0.0,
                max_tokens=500,
            )
    summary_dict[filename] = response.choices[0].message.content

Processing file: resume_sample.pdf
Processing file: resume_sample copy.pdf


In [144]:
folder_processed ="resumes/processed"

for filename in summary_dict.keys():
    with open(f"{folder_processed}/{filename}.json","a") as f:
        data = json.dump(summary_dict[filename],f , indent=4)